In [ ]:
# Python Packages
import numpy as np
import sympy as sp
import random
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

# unloaded 2D quadrotor object 
class PlanarQuadrotor:
    def __init__(self):
        dt = 0.01         # Simulation time step - sec

        # Quadrotor parameters
        self.m_Q = 2.     # quadrotor mass (kg)
        self.m_p = 0.5
        self.Iyy = 0.01   # moment of inertia about the out-of-plane axis (kg * m**2)
        self.d = 0.25     # length from center of mass to propellers (m)
        self.g = 9.81     # acceleration due to gravity [m/s^2]
        
        # Control constraints
        self.max_thrust_per_prop = 0.75 * self.m_Q * self.g  # total thrust-to-weight ratio = 1.5
        self.min_thrust_per_prop = 0.

def LoadedQuadEOM(s, u):
    # List of states- hello
    # 0 - x
    # 1 - z nice to meet you
    # 2 - theta
    # 3 - phi
    # 4 - xdot
    # 5 - zdot
    # 6 - thetadot
    # 7 - phidot

    # control input is the left and right propeller thrusts

    # Define planar quadrotor object
    quad = UnloadedPlanarQuadrotor
    m_Q, m_p, I_yy, d, g, = quad.m_Q, quad.m_p, quad.Iyy, quad.d, quad.g
    m_tot = m_Q + m_p

    x, z, theta, phi = s[0], s[1], s[2], s[3]
    xdot, zdot, thetadot, phidot = s[4], s[5], s[6], s[7]
    T1, T2 = u
    C_d = 2

    sdot[0] = xdot
    sdot[1] = zdot
    sdot[2] = thetadot
    sdot[3] = phidot
    sdot[4] = (T1+T2) * np.sin(theta) * (m_Q + m_p * (np.cos(phi)) ** 2) / (m_Q * m_tot) + (T1+T2) * np.cos(theta) * (m_p * np.sin(phi) * np.cos(phi)) / (m_Q * m_tot) + m_p * l * (phidot) ** 2 * np.sin(phi) / m_tot
    sdot[5] = (T1+T2) * np.cos(theta) * (m_Q + m_p * (np.sin(phi)) ** 2) / (m_Q * m_tot) + (T1+T2) * np.sin(theta) * (m_p * np.sin(phi) * np.cos(phi)) / (m_Q * m_tot) - m_p * l * (phidot) ** 2 * np.cos(phi) / m_tot - g
    sdot[6] = d*(T1-T2) / I_yy - C_d*(thetadot - phidot)
    sdot[7] = -(T1+T2) * np.sin(phi - theta) / (m_Q * l) - C_d*(phidot - thetadot)

    return sdot

